# Домашнее задание  № 5. Матричные разложения/Тематическое моделирование

### Задание № 1 (8 баллов)

Попробуйте матричные разложения с 4 классификаторами - SGDClassifier, KNeighborsClassifier,  RandomForest, ExtraTreesClassifier (про него подробнее почитайте в документации, он похож на RF). Используйте и NMF, SVD и LDA. Сравните результаты на кросс-валидации и выберите лучшее сочетание.

В итоге у вас должно получиться, как минимум 12 моделей (три разложения на каждый классификатор). Используйте 1 и те же параметры кросс-валидации. Параметры векторизации, параметры K в матричных разложениях, параметры классификаторов могут быть разными между экспериментами.

Можете взять поменьше данных, если все будет обучаться слишком долго (не ставьте параметр K слишком большим в NMF и LDA, иначе точно будет слишком долго)

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [32]:
import gensim
import pandas as pd
import numpy as np
from pymorphy3 import MorphAnalyzer
import pyLDAvis.gensim_models
from collections import Counter
from string import punctuation
from razdel import tokenize as razdel_tokenize
from IPython.display import Image
from IPython.core.display import HTML 
from sklearn.decomposition import TruncatedSVD, NMF, PCA, LatentDirichletAllocation
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, StratifiedKFold
from matplotlib import pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords


morph = MorphAnalyzer()
stops = stopwords.words("russian")

In [5]:
# добавим лемматизацию
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    normalized_text = [morph.parse(word)[0].normal_form for word in normalized_text]
    return ' '.join(normalized_text)


In [9]:
data = pd.read_csv('avito_category_classification.csv')

In [10]:
data

,category_name,description
0,Автомобили,"отличное состояние,обслужиание в салоне"
1,Детская одежда и обувь,В отличном состоянии. Фирма KIKO. Очень теплый...
2,Предложение услуг,"Изготовление ограждений, перил,качелей, турник..."
3,Автомобили,Автомобиль в отличном техническом состоянии. О...
4,Бытовая техника,"Продается газовая плита ""Гефест"" (Белоруссия) ..."
...,...,...
9893,Товары для детей и игрушки,Чтобы посмотреть весь ассортимент нашего магаз...
9894,Детская одежда и обувь,"Весна,осень.74-80.вопросы можно в вайбер,двухс..."
9895,"Одежда, обувь, аксессуары","Кимоно Green Hill. Состояние отличное, рост ..."
9896,Детская одежда и обувь,Б/у кроссовки на девочку. Носили только в спор...


In [11]:
data['description_norm'] = data['description'].apply(normalize)

Интерфейс у алгоритмов разложения в sklearn такой же как и у векторайзеров, только применять их нужно уже к векторизованным текстам.

In [33]:
cv = CountVectorizer(tokenizer=lambda x: x.split(), min_df=5, max_df=0.1, stop_words=stops)
svd = TruncatedSVD(500) # 500 это параметр K

In [34]:
X_sparse = cv.fit_transform(data['description_norm'])
X = svd.fit_transform(X_sparse)

Давайте посмотрим на получившиеся размерности

In [35]:
X_sparse.shape # после векторизации

(9898, 4381)

In [36]:
X.shape # после разложения

(9898, 500)

Можно посмотреть насколько сильно мы потеряли в информации (чем ближе это число к 1, тем лучше )

In [37]:
sum(svd.explained_variance_ratio_)

0.780871704608338

In [30]:
latent_squeezers = [
    TruncatedSVD,
    NMF,
    LatentDirichletAllocation
]

classifiers = [
    SGDClassifier,
    KNeighborsClassifier,
    RandomForestClassifier,
    ExtraTreesClassifier,
    MLPClassifier
]

### Задание № 2 (2 балла)

С помощью gensim постройте 5 тематических моделей с разными параметрами. Как минимум попробуйте: разное количество тем, alpha и eta параметры (в альфа попробуйте "asymmetric" и "auto" помимо 'symmetric' по дефолту, в eta просто попробуйте разные значения), а также разные параметры фильтрации словаря. 

Оцените каждую из моделей с помощью метрик (перплексии и когерентности), а также визуально, просмотрев получаемые темы. Найдите самую хорошую тему для каждой из моделей.